In [2]:
import traci
import pandas as pd
import os, sys
import sumolib
from xml.dom import minidom
import os
import random

In [3]:
sumoBinary = os.getcwd()+"/sumo/bin/sumo-gui"
config = os.getcwd()+"/sumo/tools/ChicagoTraffic/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", config]
tools = os.getcwd()+"sumo/tools"
sys.path.append(tools)
net_file = os.getcwd()+"/sumo/tools/ChicagoTraffic/osm.net.xml"

In [4]:
taxi_trips = pd.read_csv("trip_subset.csv")

In [5]:
df = taxi_trips

In [6]:
net = sumolib.net.readNet(net_file)

In [7]:
## Coordinate transformation
px = []
py = []
dx = []
dy = []
for i in df.index:
    pickup_lat = df['Pickup Centroid Latitude'].loc[i]
    pickup_lon = df['Pickup Centroid Longitude'].loc[i]
    dropoff_lat = df['Dropoff Centroid Latitude'].loc[i]
    dropoff_lon = df['Dropoff Centroid Longitude'].loc[i]
    pickup_x, pickup_y = net.convertLonLat2XY(pickup_lon, pickup_lat)
    dropoff_x, dropoff_y = net.convertLonLat2XY(dropoff_lon, dropoff_lat)
    px.append(pickup_x)
    py.append(pickup_y)
    dx.append(dropoff_x)
    dy.append(dropoff_y)

In [8]:
df['Pickup_x'] = px
df['Pickup_y'] = py
df['Dropoff_x'] = dx
df['Dropoff_y'] = dy

In [11]:
startEdge = []
endEdge = []
delta = 100
for i in df.index:
    start_x = df['Pickup_x'].loc[i]
    start_y = df['Pickup_y'].loc[i]
    end_x = df['Dropoff_x'].loc[i]
    end_y = df['Dropoff_y'].loc[i]
    startPos = [start_x, start_y]
    endPos = [end_x, end_y]
    out = sumolib.route.mapTrace([startPos, endPos], net, delta = delta)
    if len(out) == 2:
        startEdge.append(out[0].getID())
        endEdge.append(out[1].getID())
    else:
        startEdge.append(None)
        endEdge.append(None)

In [12]:
df['start_edge'] = startEdge
df['end_edge'] = endEdge

In [13]:
df = df.dropna()

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 785 entries, 3 to 1660
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Trip ID                     785 non-null    object 
 1   Taxi ID                     785 non-null    object 
 2   Trip Start Timestamp        785 non-null    object 
 3   Trip End Timestamp          785 non-null    object 
 4   Trip Seconds                785 non-null    int64  
 5   Trip Miles                  785 non-null    float64
 6   Pickup Census Tract         785 non-null    float64
 7   Dropoff Census Tract        785 non-null    float64
 8   Pickup Community Area       785 non-null    float64
 9   Dropoff Community Area      785 non-null    float64
 10  Fare                        785 non-null    float64
 11  Tips                        785 non-null    float64
 12  Tolls                       785 non-null    float64
 13  Extras                      785 no

In [15]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Pickup_x,Pickup_y,Dropoff_x,Dropoff_y,start_edge,end_edge
3,3e4087b9c6aa4c8fde67006a015dd25a41467c6c,01480513a7f6f4664d6cc4c41ff3043ae6ecbc8cb17404...,01/01/2019 08:45:00 PM,01/01/2019 08:45:00 PM,300,0.00,1.703132e+10,1.703108e+10,32.0,8.0,...,POINT (-87.6209929134 41.8849871918),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),13112.436993,10020.725181,12691.026809,11596.955148,583081972,1040118849#0
4,2d0bb399485e07bd7fed6aa4a19deb426f24f6f3,100dbc840e5381b47a92bfbdbc3f231bc18b28af8cba22...,01/02/2019 08:30:00 AM,01/02/2019 08:30:00 AM,540,1.20,1.703108e+10,1.703128e+10,8.0,28.0,...,POINT (-87.6288741572 41.8920726347),41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),12464.318153,10812.165494,11302.723054,10068.785637,965387812#7,145857394#0
7,1cc632dd348fcd9ae2b10db17b10723c9cc64c93,053e2981d702d762acc06cc5d3076f16fa3e2ad28270ac...,01/03/2019 04:30:00 PM,01/03/2019 04:30:00 PM,337,0.99,1.703108e+10,1.703108e+10,8.0,8.0,...,POINT (-87.6197106717 41.8950334495),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),13226.876501,11135.366415,11721.124693,10944.643321,461207921#0,23891801#0
11,b7feccc5630fbdad764d1372cd385b1b5b50a732,a418af5d440c9260ccfd29882c6c4ea78d7269df7d5130...,01/04/2019 04:30:00 PM,01/04/2019 04:45:00 PM,480,1.00,1.703184e+10,1.703108e+10,32.0,8.0,...,POINT (-87.6327464887 41.8809944707),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),12134.007981,9584.547444,12685.276813,10858.865207,134319745#0,56720835#3
14,63feadf057d3ae9a2ad9cd95767165e31a4e0ca8,f7c942487c3ce86530856b5920cd4f28b0cff930a8bf7e...,01/05/2019 05:00:00 PM,01/05/2019 05:00:00 PM,720,2.40,1.703108e+10,1.703184e+10,8.0,32.0,...,POINT (-87.6207628651 41.8983317935),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),13142.242392,11502.204498,12134.007981,9584.547444,-316912742,134319745#0


In [20]:
random.seed(42)
id = []
depart_times = []
ride_counter = 0
depart_time = 0
for i in df.index:
    id.append(f'taxi_{ride_counter}')
    depart_times.append(depart_time)
    ride_counter += 1
    depart_time += random.uniform(0,9)


In [21]:
df['id'] = id
df['depart'] = depart_times

/var/folders/zd/t0wwttzn6rj0h_mjd4m2_vzw0000gn/T/ipykernel_11354/3616201796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = id
/var/folders/zd/t0wwttzn6rj0h_mjd4m2_vzw0000gn/T/ipykernel_11354/3616201796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['depart'] = depart_times


In [22]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Dropoff Centroid Longitude,Dropoff Centroid Location,Pickup_x,Pickup_y,Dropoff_x,Dropoff_y,start_edge,end_edge,id,depart
3,3e4087b9c6aa4c8fde67006a015dd25a41467c6c,01480513a7f6f4664d6cc4c41ff3043ae6ecbc8cb17404...,01/01/2019 08:45:00 PM,01/01/2019 08:45:00 PM,300,0.00,1.703132e+10,1.703108e+10,32.0,8.0,...,-87.626211,POINT (-87.6262105324 41.8991556134),13112.436993,10020.725181,12691.026809,11596.955148,583081972,1040118849#0,taxi_0,0.000000
4,2d0bb399485e07bd7fed6aa4a19deb426f24f6f3,100dbc840e5381b47a92bfbdbc3f231bc18b28af8cba22...,01/02/2019 08:30:00 AM,01/02/2019 08:30:00 AM,540,1.20,1.703108e+10,1.703128e+10,8.0,28.0,...,-87.642808,POINT (-87.6428084655 41.8853000224),12464.318153,10812.165494,11302.723054,10068.785637,965387812#7,145857394#0,taxi_1,5.754841
7,1cc632dd348fcd9ae2b10db17b10723c9cc64c93,053e2981d702d762acc06cc5d3076f16fa3e2ad28270ac...,01/03/2019 04:30:00 PM,01/03/2019 04:30:00 PM,337,0.99,1.703108e+10,1.703108e+10,8.0,8.0,...,-87.637844,POINT (-87.6378442095 41.8932163595),13226.876501,11135.366415,11721.124693,10944.643321,461207921#0,23891801#0,taxi_2,5.979938
11,b7feccc5630fbdad764d1372cd385b1b5b50a732,a418af5d440c9260ccfd29882c6c4ea78d7269df7d5130...,01/04/2019 04:30:00 PM,01/04/2019 04:45:00 PM,480,1.00,1.703184e+10,1.703108e+10,32.0,8.0,...,-87.626215,POINT (-87.6262149064 41.8925077809),12134.007981,9584.547444,12685.276813,10858.865207,134319745#0,56720835#3,taxi_3,8.455202
14,63feadf057d3ae9a2ad9cd95767165e31a4e0ca8,f7c942487c3ce86530856b5920cd4f28b0cff930a8bf7e...,01/05/2019 05:00:00 PM,01/05/2019 05:00:00 PM,720,2.40,1.703108e+10,1.703184e+10,8.0,32.0,...,-87.632746,POINT (-87.6327464887 41.8809944707),13142.242392,11502.204498,12134.007981,9584.547444,-316912742,134319745#0,taxi_4,10.464098


In [23]:
df.depart.max()

3544.2922462243614

In [24]:
traffic_path = os.getcwd() + '/sumo/tools/ChicagoTraffic/osm.passenger.trips.xml'
traffic_df = pd.read_xml(traffic_path)

In [25]:
traffic_df.drop(labels = 0, axis = 0, inplace = True)

In [26]:
traffic_df.head()

,id,vClass,depart,from,to,departLane,type,departSpeed
1,veh0,None,0.00,373852574#0,-146020062#6,best,veh_passenger,None
2,veh1,None,0.08,-498999919#9,-435384628#1,best,veh_passenger,None
3,veh2,None,0.16,435372250#0,433246683#0,best,veh_passenger,None
4,veh3,None,0.23,617082820,-24096758#2,best,veh_passenger,None
5,veh4,None,0.31,194872029#9,-189755246#14,best,veh_passenger,None


In [27]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Dropoff Centroid Longitude,Dropoff Centroid Location,Pickup_x,Pickup_y,Dropoff_x,Dropoff_y,start_edge,end_edge,id,depart
3,3e4087b9c6aa4c8fde67006a015dd25a41467c6c,01480513a7f6f4664d6cc4c41ff3043ae6ecbc8cb17404...,01/01/2019 08:45:00 PM,01/01/2019 08:45:00 PM,300,0.00,1.703132e+10,1.703108e+10,32.0,8.0,...,-87.626211,POINT (-87.6262105324 41.8991556134),13112.436993,10020.725181,12691.026809,11596.955148,583081972,1040118849#0,taxi_0,0.000000
4,2d0bb399485e07bd7fed6aa4a19deb426f24f6f3,100dbc840e5381b47a92bfbdbc3f231bc18b28af8cba22...,01/02/2019 08:30:00 AM,01/02/2019 08:30:00 AM,540,1.20,1.703108e+10,1.703128e+10,8.0,28.0,...,-87.642808,POINT (-87.6428084655 41.8853000224),12464.318153,10812.165494,11302.723054,10068.785637,965387812#7,145857394#0,taxi_1,5.754841
7,1cc632dd348fcd9ae2b10db17b10723c9cc64c93,053e2981d702d762acc06cc5d3076f16fa3e2ad28270ac...,01/03/2019 04:30:00 PM,01/03/2019 04:30:00 PM,337,0.99,1.703108e+10,1.703108e+10,8.0,8.0,...,-87.637844,POINT (-87.6378442095 41.8932163595),13226.876501,11135.366415,11721.124693,10944.643321,461207921#0,23891801#0,taxi_2,5.979938
11,b7feccc5630fbdad764d1372cd385b1b5b50a732,a418af5d440c9260ccfd29882c6c4ea78d7269df7d5130...,01/04/2019 04:30:00 PM,01/04/2019 04:45:00 PM,480,1.00,1.703184e+10,1.703108e+10,32.0,8.0,...,-87.626215,POINT (-87.6262149064 41.8925077809),12134.007981,9584.547444,12685.276813,10858.865207,134319745#0,56720835#3,taxi_3,8.455202
14,63feadf057d3ae9a2ad9cd95767165e31a4e0ca8,f7c942487c3ce86530856b5920cd4f28b0cff930a8bf7e...,01/05/2019 05:00:00 PM,01/05/2019 05:00:00 PM,720,2.40,1.703108e+10,1.703184e+10,8.0,32.0,...,-87.632746,POINT (-87.6327464887 41.8809944707),13142.242392,11502.204498,12134.007981,9584.547444,-316912742,134319745#0,taxi_4,10.464098


In [28]:
df['vClass'] = None
df['departLane'] = 'best'
df['type'] = 'veh_passenger'
df['departSpeed'] = 'None'

/var/folders/zd/t0wwttzn6rj0h_mjd4m2_vzw0000gn/T/ipykernel_11354/3730449995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vClass'] = None
/var/folders/zd/t0wwttzn6rj0h_mjd4m2_vzw0000gn/T/ipykernel_11354/3730449995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['departLane'] = 'best'
/var/folders/zd/t0wwttzn6rj0h_mjd4m2_vzw0000gn/T/ipykernel_11354/3730449995.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [29]:
traffic_df.columns

Index(['id', 'vClass', 'depart', 'from', 'to', 'departLane', 'type',
       'departSpeed'],
      dtype='object')

In [30]:
df.to_csv("trip_subset_with_SUMO_ids_for_traffic.csv")

In [31]:
df2 = df[['id', 'vClass', 'depart', 'start_edge', 'end_edge', 'departLane', 'type',
       'departSpeed']]

In [32]:
df2.columns = ['id', 'vClass', 'depart', 'from', 'to', 'departLane', 'type',
       'departSpeed']

In [33]:
df2.head()

,id,vClass,depart,from,to,departLane,type,departSpeed
3,taxi_0,None,0.000000,583081972,1040118849#0,best,veh_passenger,None
4,taxi_1,None,5.754841,965387812#7,145857394#0,best,veh_passenger,None
7,taxi_2,None,5.979938,461207921#0,23891801#0,best,veh_passenger,None
11,taxi_3,None,8.455202,134319745#0,56720835#3,best,veh_passenger,None
14,taxi_4,None,10.464098,-316912742,134319745#0,best,veh_passenger,None


In [34]:
tt_df = traffic_df.append(df2)

/var/folders/zd/t0wwttzn6rj0h_mjd4m2_vzw0000gn/T/ipykernel_11354/62990112.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tt_df = traffic_df.append(df2)


In [35]:
tt_df.sort_values(by = 'depart', ascending = True, inplace = True)
tt_df.reset_index(drop = True, inplace = True)

In [36]:
tt_df.head()

,id,vClass,depart,from,to,departLane,type,departSpeed
0,veh0,None,0.00,373852574#0,-146020062#6,best,veh_passenger,None
1,taxi_0,None,0.00,583081972,1040118849#0,best,veh_passenger,None
2,veh1,None,0.08,-498999919#9,-435384628#1,best,veh_passenger,None
3,veh2,None,0.16,435372250#0,433246683#0,best,veh_passenger,None
4,veh3,None,0.23,617082820,-24096758#2,best,veh_passenger,None


In [37]:
root = minidom.Document()
xml = root.createElement('routes')
root.appendChild(xml)
for i in tt_df.index:
    start = tt_df['from'].loc[i]
    end = tt_df['to'].loc[i]
    id = tt_df['id'].loc[i]
    depart = tt_df['depart'].loc[i]
    depart_speed = tt_df['departSpeed'].loc[i]
    tripChild = root.createElement('trip')
    tripChild.setAttribute('id',f'{id}')
    tripChild.setAttribute('type','veh_passenger')
    tripChild.setAttribute('depart',f"{depart:.2f}")
    tripChild.setAttribute('departLane','best')
    tripChild.setAttribute('from',f"{start}")
    tripChild.setAttribute('to',f"{end}")
    if 'taxi' in id:
        tripChild.setAttribute('speedFactor','1')
    xml.appendChild(tripChild)
xml_str = root.toprettyxml(indent="\t")
file_path = 'taxi_trip_subset_with_traffic.xml'
with open(file_path, "w") as f:
    f.write(xml_str)